In [1]:
import pandas as pd
import numpy
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical
import tensorflow
tensorflow.keras.__version__

'2.4.0'

# Read the CSV and Perform Basic Data Cleaning

In [2]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [3]:
df.columns

Index(['koi_disposition', 'koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co',
       'koi_fpflag_ec', 'koi_period', 'koi_period_err1', 'koi_period_err2',
       'koi_time0bk', 'koi_time0bk_err1', 'koi_time0bk_err2', 'koi_impact',
       'koi_impact_err1', 'koi_impact_err2', 'koi_duration',
       'koi_duration_err1', 'koi_duration_err2', 'koi_depth', 'koi_depth_err1',
       'koi_depth_err2', 'koi_prad', 'koi_prad_err1', 'koi_prad_err2',
       'koi_teq', 'koi_insol', 'koi_insol_err1', 'koi_insol_err2',
       'koi_model_snr', 'koi_tce_plnt_num', 'koi_steff', 'koi_steff_err1',
       'koi_steff_err2', 'koi_slogg', 'koi_slogg_err1', 'koi_slogg_err2',
       'koi_srad', 'koi_srad_err1', 'koi_srad_err2', 'ra', 'dec',
       'koi_kepmag'],
      dtype='object')

# Select your features (columns)

In [4]:
# Set features. This will also be used as your x values.
X = df.drop("koi_disposition", axis=1)

# Create a Train Test Split

Use `koi_disposition` for the y values

In [5]:
y=df["koi_disposition"]
print(X.shape, y.shape)

(6991, 40) (6991,)


In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1)

# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [7]:
# Scale your data
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)

In [8]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [9]:
# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [10]:
# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train,3)
y_test_categorical = to_categorical(encoded_y_test)

# Train the Model



In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [12]:
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=X_train.shape[1]))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=3, activation='softmax'))

In [13]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               4100      
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 303       
Total params: 14,503
Trainable params: 14,503
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=100,
    shuffle=True,
    verbose=2
)

Epoch 1/100
164/164 - 1s - loss: 0.5876 - accuracy: 0.7198
Epoch 2/100
164/164 - 0s - loss: 0.3839 - accuracy: 0.7971
Epoch 3/100
164/164 - 0s - loss: 0.3641 - accuracy: 0.8198
Epoch 4/100
164/164 - 0s - loss: 0.3536 - accuracy: 0.8238
Epoch 5/100
164/164 - 0s - loss: 0.3497 - accuracy: 0.8272
Epoch 6/100
164/164 - 0s - loss: 0.3391 - accuracy: 0.8365
Epoch 7/100
164/164 - 0s - loss: 0.3403 - accuracy: 0.8354
Epoch 8/100
164/164 - 0s - loss: 0.3273 - accuracy: 0.8470
Epoch 9/100
164/164 - 0s - loss: 0.3294 - accuracy: 0.8446
Epoch 10/100
164/164 - 0s - loss: 0.3236 - accuracy: 0.8482
Epoch 11/100
164/164 - 0s - loss: 0.3209 - accuracy: 0.8537
Epoch 12/100
164/164 - 0s - loss: 0.3132 - accuracy: 0.8543
Epoch 13/100
164/164 - 0s - loss: 0.3141 - accuracy: 0.8568
Epoch 14/100
164/164 - 0s - loss: 0.3138 - accuracy: 0.8514
Epoch 15/100
164/164 - 0s - loss: 0.3020 - accuracy: 0.8652
Epoch 16/100
164/164 - 0s - loss: 0.3064 - accuracy: 0.8604
Epoch 17/100
164/164 - 0s - loss: 0.3039 - accura

In [16]:
 model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

55/55 - 0s - loss: 0.2644 - accuracy: 0.8884
Normal Neural Network - Loss: 0.2643960416316986, Accuracy: 0.8884439468383789


# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [17]:
# Create the GridSearchCV model
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
#param_grid = {'epochs': [80, 150,300]}
#scorers = {'accuracy_score': make_scorer(accuracy_score)}
#grid = GridSearchCV(model, param_grid, verbose=3, scoring=scorers, refit="accuracy_score")
#grid_result = grid.fit(X_train_scaled,y_train_categorical)


In [22]:
def getModel(optimizer):
    model1 = Sequential()
    model1.add(Dense(units=200, activation='relu', input_dim=X_train.shape[1]))
    model1.add(Dense(units=200, activation='relu'))
    model1.add(Dense(units=3, activation='softmax'))
    model1.compile(optimizer=optimizer , loss = 'categorical_crossentropy' , metrics=['accuracy'])
    return model1

#optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
optimizer=['Adam']
epochs = [200]

param_grid = dict(epochs=epochs, optimizer=optimizer)
Kmodel = KerasClassifier(build_fn=getModel, verbose=2)
grid = GridSearchCV(estimator=Kmodel, param_grid=param_grid, scoring='accuracy', n_jobs=-1, refit='accuracy_score')

In [23]:
# Train the model with GridSearch
grid.fit(X_train_scaled, y_train)


Epoch 1/200
164/164 - 1s - loss: 0.5030 - accuracy: 0.7500
Epoch 2/200
164/164 - 0s - loss: 0.3746 - accuracy: 0.8117
Epoch 3/200
164/164 - 0s - loss: 0.3584 - accuracy: 0.8180
Epoch 4/200
164/164 - 0s - loss: 0.3474 - accuracy: 0.8255
Epoch 5/200
164/164 - 0s - loss: 0.3353 - accuracy: 0.8375
Epoch 6/200
164/164 - 0s - loss: 0.3361 - accuracy: 0.8377
Epoch 7/200
164/164 - 0s - loss: 0.3236 - accuracy: 0.8516
Epoch 8/200
164/164 - 0s - loss: 0.3181 - accuracy: 0.8514
Epoch 9/200
164/164 - 0s - loss: 0.3093 - accuracy: 0.8619
Epoch 10/200
164/164 - 0s - loss: 0.3131 - accuracy: 0.8562
Epoch 11/200
164/164 - 0s - loss: 0.3080 - accuracy: 0.8638
Epoch 12/200
164/164 - 0s - loss: 0.3047 - accuracy: 0.8667
Epoch 13/200
164/164 - 0s - loss: 0.2965 - accuracy: 0.8707
Epoch 14/200
164/164 - 0s - loss: 0.2969 - accuracy: 0.8682
Epoch 15/200
164/164 - 0s - loss: 0.2964 - accuracy: 0.8657
Epoch 16/200
164/164 - 0s - loss: 0.2919 - accuracy: 0.8680
Epoch 17/200
164/164 - 0s - loss: 0.2935 - accura

GridSearchCV(estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x00000179A29FEA90>,
             n_jobs=-1, param_grid={'epochs': [200], 'optimizer': ['Adam']},
             refit='accuracy_score', scoring='accuracy')

In [24]:
print(grid.best_params_)
print(grid.best_score_)

{'epochs': 200, 'optimizer': 'Adam'}
0.8821252883516835


# Save the Model

In [21]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
model.save("deep_learning.h5")